# Part I: Understanding the Problem and Data Exploration
<img src="IMG/workflow.png">

## Frame the problem

Before looking at the data it is important to understand how does the company expect to use and benefit from this model? This first brainstorming helps to determine how to frame the problem, what algorithms to select and measure the performance of each one.

## Make Assumptions

It is good to make some assumptions on what possible outcomes we might expect from our analysis according to the available data. Therefore, by knowing the goal we should think which possible factors might affect the sales prediction outcome.

# Start Data Exploration (EDA)
* browse and clean data
* look univariate and multivariate statistics
* isolate and investigate the statistics of groups
* visualize interesting findings
* prepare a small report on your findings (in a seperate notebook)

## Getting Started


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # Ignores any warning
warnings.filterwarnings("ignore")

#Important. for now, we only look at the available train data
train = pd.read_csv("DATA/Train.csv") 


In [61]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [62]:
train.isnull().any()


Item_Identifier              False
Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
dtype: bool

In [63]:
train_nans = train.isnull()
train_nans = train_nans.any(axis=1)
train_nans


0       False
1       False
2       False
3        True
4       False
5       False
6       False
7        True
8        True
9        True
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18       True
19      False
20      False
21       True
22      False
23       True
24      False
25       True
26      False
27      False
28       True
29       True
        ...  
8493     True
8494     True
8495    False
8496     True
8497    False
8498    False
8499    False
8500     True
8501     True
8502     True
8503    False
8504     True
8505    False
8506    False
8507    False
8508     True
8509     True
8510    False
8511    False
8512    False
8513    False
8514     True
8515    False
8516    False
8517    False
8518    False
8519     True
8520    False
8521    False
8522    False
Length: 8523, dtype: bool

In [64]:
# Erkenntnis: Es gibt nur nans in weigth und size --> nummerische Attribute --> mathematische Ansätze (median/mean) zur Ersetzung von nans möglich
# Idee: Korrelation zwischen weight und type rauszufinden und dann ggf. 
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000


In [65]:
def fat_to_numeric(x):
    if (x == 'Low Fat') or (x == 'low fat') or (x == 'LF'):
        return 0
    if (x == 'Regular') or (x == 'reg'):
        return 1
    
def type_to_numeric(x):
    if (x == 'Dairy'):
        return 0
    if (x == 'Soft Drinks'):
        return 1
    if (x == 'Meat'):
        return 2
    if (x == 'Fruits and Vegetables'):
        return 3
    if (x == 'Household'):
        return 4
    if (x == 'Baking Goods'):
        return 5
    if (x == 'Snack Foods'):
        return 6
    if (x == 'Frozen Foods'):
        return 7
    if (x == 'Breakfast'):
        return 8
    if (x == 'Health and Hygiene'):
        return 9
    if (x == 'Hard Drinks'):
        return 10
    if (x == 'Canned'):
        return 11
    if (x == 'Breads'):
        return 12
    if (x == 'Starchy Foods'):
        return 13
    if (x == 'Others'):
        return 14
    if (x == 'Seafood'):
        return 15
    
def size_to_numeric(x):
    if (x == 'Small'):
        return 0
    if (x == 'Medium'):
        return 1
    if (x == 'High'):
        return 2
    return 3
    
def loc_type_to_numeric(x):
    if (x=='Tier 1'):
        return 0
    if (x=='Tier 2'):
        return 1
    if (x=='Tier 3'):
        return 2

def type_to_numeric(x):
    if (x=='Supermarket Type1'):
        return 0
    if (x=='Supermarket Type2'):
        return 1
    if (x=='Supermarket Type3'):
        return 2
    if (x=='Grocery Store'):
        return 3
    
    

In [66]:
train['Item_Type_n'] = train['Item_Type'].apply(type_to_numeric)

In [67]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000


In [68]:
train['Item_Fat_Content_n'] = train['Item_Fat_Content'].apply(fat_to_numeric)

In [69]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_n
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123,-0.023238
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625,0.047314
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574,0.006063
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135,0.003151
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000,0.018719
Item_Fat_Content_n,-0.023238,0.047314,0.006063,0.003151,0.018719,1.000000


In [70]:
types = train.groupby('Item_Type')# ["Item_Wight"]
for grp, data in types:
    print(grp+ ':')
    print('mean: ' + str(data['Item_Weight'].mean()))
    print('median: ' + str(data['Item_Weight'].median()))
    print( ' ')

Baking Goods:
mean: 12.277108208955255
median: 11.65
 
Breads:
mean: 11.3469362745098
median: 10.6
 
Breakfast:
mean: 12.768202247191002
median: 10.695
 
Canned:
mean: 12.305705009276451
median: 12.15
 
Dairy:
mean: 13.42606890459367
median: 13.35
 
Frozen Foods:
mean: 12.867061281337076
median: 12.85
 
Fruits and Vegetables:
mean: 13.224769381746881
median: 13.1
 
Hard Drinks:
mean: 11.400327868852452
median: 10.1
 
Health and Hygiene:
mean: 13.142313953488392
median: 12.15
 
Household:
mean: 13.384736495388726
median: 13.15
 
Meat:
mean: 12.81734421364986
median: 12.35
 
Others:
mean: 13.853284671532842
median: 14.5
 
Seafood:
mean: 12.5528431372549
median: 11.65
 
Snack Foods:
mean: 12.987879554655919
median: 13.15
 
Soft Drinks:
mean: 11.847459893048129
median: 11.8
 
Starchy Foods:
mean: 13.690730769230765
median: 13.175
 


In [71]:
# --> meadian und mean liegen relativ nahe beisammen und zwar bei allen Typen --> keine einseitigen Ausreiser --> für nans in weght median/ mean benutzen

In [72]:
train['Item_Weight'].fillna(train['Item_Weight'].mean() ,inplace=True)

In [73]:
# gleiches noch für outletsize
types = train.groupby('Item_Type')# ["Item_Wight"]
for grp, data in types:
    print(grp+ ':')
    print('mean: ' + str(data['Item_Weight'].mean()))
    print('median: ' + str(data['Item_Weight'].median()))
    print( ' ')

Baking Goods:
mean: 12.377447933060623
median: 12.857645184136183
 
Breads:
mean: 11.629818022527507
median: 12.8
 
Breakfast:
mean: 12.785277716971441
median: 12.857645184136183
 
Canned:
mean: 12.399254191456087
median: 12.857645184136183
 
Dairy:
mean: 13.329386864163975
median: 12.857645184136183
 
Frozen Foods:
mean: 12.865543265666867
median: 12.857645184136183
 
Fruits and Vegetables:
mean: 13.161297422257393
median: 12.857645184136183
 
Hard Drinks:
mean: 11.6114345827487
median: 11.225000000000001
 
Health and Hygiene:
mean: 13.093044358792845
median: 12.857645184136183
 
Household:
mean: 13.297274090994096
median: 12.857645184136183
 
Meat:
mean: 12.825688885185887
median: 12.857645184136183
 
Others:
mean: 13.664761218297972
median: 12.857645184136183
 
Seafood:
mean: 12.614756053027666
median: 12.728822592068092
 
Snack Foods:
mean: 12.964871482530803
median: 12.857645184136183
 
Soft Drinks:
mean: 12.008635523761084
median: 12.857645184136183
 
Starchy Foods:
mean: 13.5894

In [74]:
train['Outlet_Size_n'] = train['Outlet_Size'].apply(size_to_numeric)

In [75]:
train['Outlet_Location_Type_n'] = train['Outlet_Location_Type'].apply(loc_type_to_numeric)

In [76]:
train['Outlet_Type_n'] = train['Outlet_Type'].apply(type_to_numeric)

In [77]:
train['Outlet_Location_Type_n'] = train['Outlet_Location_Type'].apply(loc_type_to_numeric) 

In [78]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_n,Outlet_Size_n,Outlet_Location_Type_n,Outlet_Type_n
Item_Weight,1.000000,-0.012049,0.024756,-0.008301,0.011550,-0.021157,-0.005356,0.004088,0.002725
Item_Visibility,-0.012049,1.000000,-0.001315,-0.074834,-0.128625,0.047314,0.003108,-0.028099,0.223308
Item_MRP,0.024756,-0.001315,1.000000,0.005020,0.567574,0.006063,-0.008320,0.000232,-0.006671
Outlet_Establishment_Year,-0.008301,-0.074834,0.005020,1.000000,-0.049135,0.003151,0.199485,-0.089216,-0.433070
Item_Outlet_Sales,0.011550,-0.128625,0.567574,-0.049135,1.000000,0.018719,-0.057464,0.089367,-0.209854
Item_Fat_Content_n,-0.021157,0.047314,0.006063,0.003151,0.018719,1.000000,-0.000517,-0.001598,-0.001967
Outlet_Size_n,-0.005356,0.003108,-0.008320,0.199485,-0.057464,-0.000517,1.000000,0.385597,-0.059041
Outlet_Location_Type_n,0.004088,-0.028099,0.000232,-0.089216,0.089367,-0.001598,0.385597,1.000000,0.290939
Outlet_Type_n,0.002725,0.223308,-0.006671,-0.433070,-0.209854,-0.001967,-0.059041,0.290939,1.000000


In [79]:
# ==> Korrelation zwischen Size und Type angenommen
sizes = train.groupby(['Outlet_Type', 'Outlet_Size']).size()
sizes

Outlet_Type        Outlet_Size
Grocery Store      Small           528
Supermarket Type1  High            932
                   Medium          930
                   Small          1860
Supermarket Type2  Medium          928
Supermarket Type3  Medium          935
dtype: int64

In [80]:
train.groupby('Outlet_Type').get_group('Grocery Store')['Outlet_Size'].count()

528

In [81]:
# == > gruppierung hat funktioniert

In [90]:
for index, row in train.iterrows():
    if (row['Outlet_Size_n']== 3):
        if (row['Outlet_Type'] == 'Grocery Store') or (row['Outlet_Type'] == 'Supermarket Type1'):
            train.at[index, 'Outlet_Size'] = 'Small'
        if (row['Outlet_Type'] == 'Supermarket Type2') or (row['Outlet_Type'] == 'Supermarket Type3'):
            train.at[index, 'Outlet_Size'] = 'Medium'


In [91]:
train.isnull().any()


Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
Item_Type_n                   True
Item_Fat_Content_n           False
Outlet_Size_n                False
Outlet_Location_Type_n       False
Outlet_Type_n                False
dtype: bool

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # Ignores any warning    

def fat_to_numeric(x):
    if (x == 'Low Fat') or (x == 'low fat') or (x == 'LF'):
        return 0
    if (x == 'Regular') or (x == 'reg'):
        return 1
    
def type_to_numeric(x):
    if (x == 'Dairy'):
        return 0
    if (x == 'Soft Drinks'):
        return 1
    if (x == 'Meat'):
        return 2
    if (x == 'Fruits and Vegetables'):
        return 3
    if (x == 'Household'):
        return 4
    if (x == 'Baking Goods'):
        return 5
    if (x == 'Snack Foods'):
        return 6
    if (x == 'Frozen Foods'):
        return 7
    if (x == 'Breakfast'):
        return 8
    if (x == 'Health and Hygiene'):
        return 9
    if (x == 'Hard Drinks'):
        return 10
    if (x == 'Canned'):
        return 11
    if (x == 'Breads'):
        return 12
    if (x == 'Starchy Foods'):
        return 13
    if (x == 'Others'):
        return 14
    if (x == 'Seafood'):
        return 15
    
def size_to_numeric(x):
    if (x == 'Small'):
        return 0
    if (x == 'Medium'):
        return 1
    if (x == 'High'):
        return 2
    return 3
    
def loc_type_to_numeric(x):
    if (x=='Tier 1'):
        return 0
    if (x=='Tier 2'):
        return 1
    if (x=='Tier 3'):
        return 2

def type_to_numeric(x):
    if (x=='Supermarket Type1'):
        return 0
    if (x=='Supermarket Type2'):
        return 1
    if (x=='Supermarket Type3'):
        return 2
    if (x=='Grocery Store'):
        return 3

def proc_data(path):
    df = pd.read_csv(path)
    df['Item_Type_n'] = df['Item_Type'].apply(type_to_numeric)
    df['Item_Fat_Content_n'] = df['Item_Fat_Content'].apply(fat_to_numeric)
    
    df['Item_Weight'].fillna(df['Item_Weight'].mean() ,inplace=True)
    
    df['Outlet_Size_n'] = df['Outlet_Size'].apply(size_to_numeric)
    
    df['Outlet_Location_Type_n'] = df['Outlet_Location_Type'].apply(loc_type_to_numeric)
    
    df['Outlet_Type_n'] = df['Outlet_Type'].apply(type_to_numeric)
    
    df['Outlet_Location_Type_n'] = df['Outlet_Location_Type'].apply(loc_type_to_numeric) 
    
    for index, row in df.iterrows():
        if (row['Outlet_Size_n']== 3):
            if (row['Outlet_Type'] == 'Grocery Store') or (row['Outlet_Type'] == 'Supermarket Type1'):
                df.at[index, 'Outlet_Size'] = 'Small'
            if (row['Outlet_Type'] == 'Supermarket Type2') or (row['Outlet_Type'] == 'Supermarket Type3'):
                df.at[index, 'Outlet_Size'] = 'Medium'
    return df
    
    

In [4]:
df = proc_data("DATA/Train.csv")

In [6]:
df.isnull().any()


Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
Item_Type_n                   True
Item_Fat_Content_n           False
Outlet_Size_n                False
Outlet_Location_Type_n       False
Outlet_Type_n                False
dtype: bool

In [3]:
import data_prep

In [4]:
df = data_prep.proc_data("DATA/Train.csv")

In [5]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Type_n,Item_Fat_Content_n,Outlet_Size_n,Outlet_Location_Type_n,Outlet_Type_n
0,FDA15,9.300000,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,None,0,1,0,0
1,DRC01,5.920000,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,None,1,1,2,1
2,FDN15,17.500000,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,None,0,1,0,0
3,FDX07,19.200000,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,None,1,3,2,3
4,NCD19,8.930000,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,None,0,2,2,0
5,FDP36,10.395000,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,None,1,1,2,1
6,FDO10,13.650000,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,None,1,2,2,0
7,FDP10,12.857645,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,None,0,1,2,2
8,FDH17,16.200000,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986,None,1,3,1,0
9,FDU28,19.200000,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350,None,1,3,1,0
